# 1. Introdução

# 2. Obtenção do dataset

## 2.1. Imagens dos pokemons

No inicio do desenvolvimento do projeto, o maior problema enfrentado foi justamente encontrar imagens de cada um dos pokemons. Isso é um problema visto que para treinar o modelo, precisaríamos de bem mais que uma imagem de cada um deles, e encontrar um banco de dados com essas imagens diretamente não foi possível.

Surgiu então a ideia de utilizar as imagens utilizadas como arte das cartas do jogo de carta tematizado no universo de pokemon. Isso foi conseguido por meio de um web scraper desenvolvido para percorrer as páginas do website https://pkmncards.com/, que é um site onde jogadores do jogo de carta podem visitar para olhar informações sobre cada uma das cartas do jogo.

Cada uma dessas cartas possuí uma arte diferente, mesmo sendo do mesmo pokemon. Isso implica que caso pudessemos utilizar essas artes para treinar o modelo, teríamos uma quantidade muito maior de dados de entrada do que usando apenas uma imagem para cada um dos pokemons.

<table>
<tr>
<td><img src='https://i.imgur.com/6BeJaIB.jpg'/ style="height: 250px;"></td>
<td><img src='https://i.imgur.com/aDqF2nn.jpg'/ style="height: 250px;"></td>
<td><img src='https://i.imgur.com/G1xLJcc.jpg'/ style="height: 250px;"></td>
</tr>
</table>

Pode ser observado também, que em geral, a imagem do pokemon fica em uma posição bem específica em cada uma das cartas. Isso nos possibilita obter a imagem dos pokemons de forma fácil. O resultado é como nas imagens a seguir.

<table>
<tr>
<td><img src='https://i.imgur.com/ainlu5d.jpg'/ style="height: 150px;"></td>
<td><img src='https://i.imgur.com/pWPpPle.jpg'/ style="height: 150px;"></td>
<td><img src='https://i.imgur.com/FD5l5v1.jpg'/ style="height: 150px;"></td>
</tr>
</table>

Porém existem cartas especiais chamadas de "full cover", onde a arte do pokemon ocupa todo o espaço da carta. Como não havia um jeito prático de separar essas cartas das cartas comuns, elas ficaram como um ruído no dataset. Esse ruído não é tão ruim já que geralmente em cartas full cover, a "face" do pokemon fica no lugar onde a imagem fica em cartas comum, então a imagem gerada após cortar a região da carta ainda faz algum sentido. Alguns exemplos de cartas full cover cortadas vem a seguir.

<table>
<tr>
<td><img src='https://i.imgur.com/Y7AiT7L.jpg'/ style="height: 150px;"></td>
<td><img src='https://i.imgur.com/umSI8lZ.jpg'/ style="height: 150px;"></td>
</tr>
</table>

Ao final, foram obtidas 10455 imagens de 807 pokemons diferentes.

## 2.2. Tipos de cada pokemon

Os tipos de pokemon foram obtidos a partir de um CSV disponibilizado pelo projeto no github do usuário Veekun. http://github.com/veekun/pokedex/

<table>
<tr>
<td><img src='https://i.imgur.com/JZkB2F5.jpg'/ style="height: 250px;"></td>
</tr>
</table>

Esse CSV tem, em cada linha, o ID de um pokemon, o ID de um dos seus tipo e uma variável ```slot``` que indica se aquele tipo é o primeiro ou o segundo tipo do pokemon em questão.

# 3. Implementação de funções auxiliares

O método abaixo serve para auxiliar posteriormente a plotar métricas do modelo treinado.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def pretty_plot(all_metrics, metric_info, texts, x_tick_labels):
    stats = []
    for m in metric_info:
        stats.append(np.array([s[m["name"]] for s in all_metrics]))
        
    xs = np.linspace(0, stats[0].shape[0] - 1, stats[0].shape[0])
    
    errorbar_style = {"linestyle":"--", "linewidth":1, "markeredgewidth":2, "elinewidth":2, "capsize":3, "marker": "^"}
    plot_style = {"linestyle":"--", "linewidth":1, "markeredgewidth":2, "marker": "^"}
    
    fig = plt.figure(figsize=(12, 8))

    ax = [plt.subplot(2, 1, i + 1) for i in range(len(metric_info))]
    
    colors = ["#e74c3c", "#2ecc71"]
    
    for i, m in enumerate(metric_info):
        curr_color = colors[i % len(colors)]
        if m["type"] == "error":               
            ax[i].errorbar(xs, stats[i].mean(axis=1), stats[i].std(axis=1), **errorbar_style, color=curr_color)
        elif m["type"] == "simple":
            if m["format"] == "mean":
                ax[i].plot(xs, stats[i].mean(axis=1), **plot_style, color=curr_color)
            elif m["format"] == "single":
                ax[i].plot(xs, stats[i], **plot_style, color=curr_color)
             
        ax[i].xaxis.set_ticks(xs)
        ax[i].set_xticklabels(x_tick_labels)
        ax[i].set_title(texts[i]["title"])
        ax[i].set_xlabel("Fold")
        ax[i].set_ylabel(texts[i]["ylabel"])

    plt.tight_layout()
    plt.show()

# 4. Implementação do algoritmo de aprendizado

## 4.1. Definição dos tipos

Inicialmente, iremos definir uma das constantes mais importantes do problema: os tipos de Pokemon existentes.

Atualmente, existem 18 tipos Pokemon:
 - Normal, Lutador, Voador, Venenoso, Terra, Pedra, Inseto, Fantasma, Metálico, Fogo, Água, Planta, Elétrico, Psíquico, Gelo, Dragrão, Noturno e Fada
 
<img src="https://i.imgur.com/sggUsfN.png" style="height: 200px;">
 
Na célula abaixo é definido uma lista com esses tipos de modo a indexa-los para uso futuro

In [ ]:
types_label = [
    "normal", "fighting", "flying", "poison", "ground", "rock",
    "bug", "ghost", "steel", "fire", "water", "grass", "electric",
    "psychic", "ice", "dragon", "dark", "fairy"
]

## 4.2. Gerenciando o dataset

Para gerenciar o dataset, foi criada uma classe ```DatasetHandler``` que tem como entrada o caminho para a pasta onde todas as imagens estão, o caminho para o arquivo CSV contendo as informações de qual pokemon tem quais tipos e o algoritmo de data augmentation que será definido posteriormente.

Essa classe tem 3 métodos:
 - ```pokemons_to_labels```
 
   - Cada Pokemon pode ter 1 ou 2 tipos associados a ele. Para obter essas informações foi obtido um arquivo CSV que contém, em cada linha, o ID de um Pokemon e o ID de um dos seus tipos. Esse ID do tipo segue a ordem de tipos definido pela variável ```types_label``` definida anteriormente.

     Com esse CSV aberto, será criado um dicionário ```types_by_pokemon``` que associa o ID de um pokemon a um N-Hot-Encoding de seu tipo (ou seja, uma lista com 18 posições onde uma posição tem o valor ```1``` caso o índice daquela posição corresponde a um tipo que aquele pokemon tem e ```0``` caso contrário).

     Então, a posição do dicionário associada ao pokemon "Bulbassauro" deverá ser ```[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]``` visto que seus tipos são Planta e Venenoso, correspondendo ás posições 3 e 11 da lista.


 - ```create_data_generators``` que é responsável por, a partir de um DataFrame contendo o caminho para uma das imagens e os valores
 
   - Esse método cria objetos ```DataGenerator``` do Keras a partir de um DataFrame contendo o caminho para as imagens do dataset e, para cada uma delas, 18 valores correspondendo ao N-Hot-Encoding criado pela função ```pokemons_to_labels```. Esses ```DataGenerators``` servirão de uso quando utilizaremos alguns métodos de transformação nas imagens obtidas.
   
   
 - ```create_dataset``` que é resoponsável por criar uma DataFrame
   - Esse método é quem junta os outros dois e retorna um dataset final. Ele cria o DataFrame que será utilizado pelo método ```create_data_generators``` a partir dos resultados do método ```pokemons_to_labels``` e ao fim retorna os ```DataGenerators``` de treino e validação.

In [ ]:
import random
from imutils import paths
import numpy as np
import pandas as pd

class DatasetHandler:
  def __init__(self, image_paths, types_csv, datagen, fold=None):
    # Load and shuffle image paths
    self.image_paths = list(paths.list_images(image_paths))
    
    self.fold = fold

    # Load types csv
    self.types = pd.read_csv(types_csv)

    # ImageGenerator options
    self.datagen = datagen

  # With the pokemon pokedex id as input, this function returns in an
  # one-hot-encoding form their encoded types
  def pokemons_to_labels(self):
    types_by_pokemon = {}
    min_id = self.types.pokemon_id.min()
    max_id = self.types[self.types.pokemon_id <= 1000].pokemon_id.max()

    for i in range(min_id, max_id + 1):
      types_id = self.types[self.types.pokemon_id == i]["type_id"].to_numpy()
      one_hot = [0] * len(types_label)
      for t_id in types_id:
        one_hot[t_id - 1] = 1

      types_by_pokemon[i] = one_hot

    return types_by_pokemon

  # Creates a DataFrame with all pokemon image paths and their correspondent
  # one-hot-encoded types.
  def create_dataset(self, verbose=False, seed=None):
    types_by_pokemon = self.pokemons_to_labels()

    paths, labels = [], []
    for path in self.image_paths:
      pkm_id = int(path.split("/")[-1].split("-")[0])
      labels.append(types_by_pokemon[pkm_id])
      paths.append(path)

    dataset_df = pd.DataFrame(labels, columns=types_label)
    dataset_df["path"] = paths

    train_gen, val_gen = self.create_data_generators(dataset_df, seed)
    return dataset_df, train_gen, val_gen

  # Creates the data generators from the dataframe inputted
  def create_data_generators(self, df_dataset, seed=None):
    dataset_cols = ["path"] + types_label

    for col in dataset_cols[1:]:
      df_dataset[col] = pd.to_numeric(df_dataset[col])

    if self.fold is None:
        train_generator = self.datagen.flow_from_dataframe(
          dataframe=df_dataset,
          x_col=dataset_cols[0],
          y_col=dataset_cols[1:],
          subset="training",
          batch_size=32,
          shuffle=True,
          class_mode="raw",
          target_size=(100, 137),
          seed=seed
        )

        valid_generator = self.datagen.flow_from_dataframe(
          dataframe=df_dataset,
          x_col=dataset_cols[0],
          y_col=dataset_cols[1:],
          subset="validation",
          batch_size=32,
          shuffle=True,
          class_mode="raw",
          target_size=(100, 137),
          seed=seed
        )
    else:
        fold_size = len(df_dataset) / self.fold[1]
        fold_init = int(self.fold[0] * fold_size)
        fold_end = int((self.fold[0] + 1) * fold_size)
        
        train_fold_i = list(range(fold_init)) + list(range(fold_end, len(df_dataset)))
        test_fold_i = list(range(fold_init, fold_end))
        train_df = df_dataset.iloc[train_fold_i]
        test_df = df_dataset.iloc[test_fold_i]
        
        train_generator = self.datagen.flow_from_dataframe(
          dataframe=train_df,
          x_col=dataset_cols[0],
          y_col=dataset_cols[1:],
          batch_size=32,
          shuffle=True,
          class_mode="raw",
          target_size=(100, 137),
          seed=seed
        )

        valid_generator = self.datagen.flow_from_dataframe(
          dataframe=test_df,
          x_col=dataset_cols[0],
          y_col=dataset_cols[1:],
          batch_size=32,
          shuffle=True,
          class_mode="raw",
          target_size=(100, 137),
          seed=seed
        )

    return train_generator, valid_generator

## 4.3. Definição do algoritmo de Data-Augmentation

Para evitar overfit, será utilizada a classe ```ImageDataGenerator``` provida pelo Keras que, a partir dos ```DataGenerators``` gerados previamente, ele fará algumas pequenas modificações nas imagens a medida que o modelo é treinado. Essas pequenas modificações incluem rotacionar as imagens, dar (ou tirar) zoom e espelha-las.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_gen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    validation_split=0.2,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest",
    rotation_range=0.2
)

## 4.4. Definição do modelo que será utilizado

Para essa tarefa, como é baseada em encontrar features em imagens, naturalmente as Redes Neurais Convolucionais se tornam uma alternativa interessante, e assim foi escolhida a arquitetura que será utilizada para resolver esse problema.

A entrada para essa CNN são as imagens dos pokemons (usandos os 3 canais de cores) e como saída esperamos o N-Hot-Encoding que nos dirá o tipo daquele pokemon.

O modelo utilizado será o ```SqueezeNet``` devido a sua simplicidade e leveza, fazendo o treinamento da rede menos custoso e mais rápido em troca de sua capacidade. A implementação desse modelo foi tirada do github do usuário ```DT42``` (https://github.com/DT42/squeezenet_demo/) e modificada para que ela funcionasse como um modelo multi-label para a predição de até 2 tipos de um pokemon.

Para o otimizador, foi escolhido o ```adam``` que é um otimizador bom para CNNs em diversos problemas. Já para a função de perda foi escolhida a ```binary_crossentropy``` visto que ela da suporte a classificação binária múltipla (mais de uma opção marcada como 1 possível).

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from model import SqueezeNet

def create_model():
    model = SqueezeNet(nb_classes=18, inputs=(100, 137, 3))
    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model

## 4.5. Crossvalidation com 5 folds

Para validar o experimento, foi feito um crossvalidation com 5 folds no dataset. Em cada iteração, $\frac{1}{5}$ dos dados diferentes das outras iterações foram utilizadas como validação enquanto as os outros $\frac{4}{5}$ dos dados foram utilizados como treino.

In [ ]:
image_path = "dataset/images/"
types_csv_path = "dataset/pokemon_types.csv"

fold_histories = []

for fold in range(5):   
    ds_handler = DatasetHandler(image_path, types_csv_path, data_gen, (fold, 5))
    df_dataset, train_generator, valid_generator = ds_handler.create_dataset(verbose=False, seed=42)
    
    model = create_model()
    callbacks = [EarlyStopping(monitor='val_loss', patience=15)]
    
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.n // train_generator.batch_size,
        validation_data=valid_generator,
        validation_steps=valid_generator.n // valid_generator.batch_size,
        epochs=500,
        verbose=1,
        callbacks=callbacks
    )
    
    fold_histories.append(history)
    
all_metrics = []
for h in fold_histories:
    all_metrics.append(h.history)

pretty_plot(all_metrics, [
    { "name": "val_loss", "type": "simple", "format": "mean" }
], [
    { "title": "Validation Binary Crossentropy Loss over 5 folds", "ylabel": "Validation Binary Crossentropy Loss" },
], list(range(1, 6)))

Os erros de validação obtidos durante o 5-fold podem ser observados na figura abaixo.

<img src="https://i.imgur.com/VfLhi4Z.png" align="left">

## 4.6. Otimização do modelo final

Foram treinados diversos modelos, variando hiperparâmetros como ```batch_size``` e parâmetros do data augmentation. Esses experimentos não só testaram combinações diferentes de valores desses hiperparâmetros, mas também deram mais oportunidades para a rede neural não cair em um mínimo local muito cedo durante o treinamento.

Para essa otimização de parâmetros foi utilizado o Grid Search, variando o batch size nos valores ```[8, 16, 32, 64]```, o máximo de zoom das imagens entre ```[0.0, 0.15, 0.3, 0.45]``` e se o algoritmo pode ou não espelhar as imagens. Os melhores resultados foram obtidos com os parâmetros

 - ```batch_size = 32```
 - ```zoom_range = 0.15```
 - ```horizontal_flip = True```
 
Um passo futuro para o desenvolvimento desse projeto seria incluir mais hiperparâmetros para serem otimizados, porém, devido ao tempo que experimentos tomam aumentarem exponencialmente com a inclusão de mais hiperparâmetros, os resultados apresentados aqui conterão apenas os citados previamente.

### Modelo Final

O modelo final que atingiu o menor ```validation loss``` conseguiu um valor de ```val_loss = 0.2503```. Exemplos dos resultados obtidos por meio da aplicação dele nos dados de validação podem ser observados a seguir.

# 5. Visualização de resultados

A célula abaixo pega os dados de validação e executa o modelo sobre eles e retorna uma lista de probabilidades para cada um dos tipos que representa a confiança do modelo de que aquele pokemon da imagem é daquele tipo.

In [ ]:
from tensorflow.keras.models import load_model

NUM_EXAMPLES = 16

ds_handler = DatasetHandler(image_path, types_csv_path, data_gen)
_, _, validation_generator = ds_handler.create_dataset(verbose=False, seed=42)

model = load_model("models/pkm_model-0.2503.hdf5")

example_count = 0
while example_count < NUM_EXAMPLES:
  x_batch, y_batch = validation_generator.next()
  for i, image in enumerate(x_batch):
    example_count += 1
    if example_count > NUM_EXAMPLES:
      break
    plt.imshow(image)
    plt.show()
    prediction = model.predict(np.expand_dims(image, axis=0))[0]
    prediction_labeled = sorted(list(zip(prediction, types_label)), reverse=True)
    for prob, name in prediction_labeled[:6]:
      print(name, prob * 100, "%")

## 5.1. Análise de resultados

Nessa seção mostrarei alguns exemplos onde o modelo acertou, errou e possíveis explicações.

Na tabela a seguir, na primeira coluna temos uma imagem que mostra o nome do pokemon em questão, uma imagem dele os tipos corretos dele nos jogos. Na coluna do meio temos a imagem dada ao modelo e embaixo da imagem temos os 6 tipos que o modelo tem mais confiança. Na última coluna temos comentários sobre a predição.

|            Pokemon Original          |     Imagem de entrada / Predições    |       Comentários      |
|:------------------------------------:|:------------------------------------:|:------------------------------------:|
| ![](https://i.imgur.com/eh1yR1G.png) | ![](https://i.imgur.com/QL0qlz0.png) | O pokemon está com um tom de azul e textura que lembra água |
| ![](https://i.imgur.com/LLiF9Y0.png) | ![](https://i.imgur.com/WglhWVB.png) | Predizeu corretamente. Porém as cores ao redor do pokemon influenciaram bastante as previsões seguintes a previsão correta |
| ![](https://i.imgur.com/YOFYEqi.png) | ![](https://i.imgur.com/kSiSLaz.png) | Muito verde geralmente indica um pokemon de planta ou inseto |
| ![](https://i.imgur.com/GxmeOLI.png) | ![](https://i.imgur.com/RAQDUC8.png) | Acertou, mas é possível perceber que como amarelo geralmente indicia um pokemon elétrico, a predição de "elétrico" ficou em segundo lugar). |
| ![](https://i.imgur.com/vBrk7p3.png) | ![](https://i.imgur.com/1ohSBbF.png) | Roxo geralmente indica um pokemon venenoso ou psíquico, mas nesse exemplo o modelo conseguiu distinguir que o pokemon não é psíquico |
| ![](https://i.imgur.com/NFTvT1k.png) | ![](https://i.imgur.com/46V6N48.png) | Não conseguiu prever o tipo elétrico, provavelmente pela ausência da cor amarela |
| ![](https://i.imgur.com/tTEokHj.png) | ![](https://i.imgur.com/Mi0YDYj.png) | Envolta do pokemon tem muitos tons de cinza, cores que geralmente estão presentes em pokemons normais ou voadores. |
| ![](https://i.imgur.com/NTjgkGo.png) | ![](https://i.imgur.com/B3JYZI9.png) | Rosa geralmente indica pokemons fada. A predição de "planta" está alta também devido à floresta |
| ![](https://i.imgur.com/wuzrsYS.png) | ![](https://i.imgur.com/SZkvZY9.png) | Claramente o modelo chutou "água" e "elétrico" por conta das cores predominantes da imagem (azul e amarelo). |

# 7. Comentários finais

